# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

# Извините, пока заглушка, 11.02 будет

In [2]:
import math
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 


In [4]:
ds = pd.read_csv('train.csv')

In [14]:

ds.iloc[3]

item_id                                                      02996f1dd2ea
user_id                                                      bf5cccea572d
region                                                          Татарстан
city                                                     Набережные Челны
parent_category_name                                          Личные вещи
category_name                                  Товары для детей и игрушки
param_1                                              Автомобильные кресла
param_2                                                               NaN
param_3                                                               NaN
title                                                          Автокресло
description                                        Продам кресло от0-25кг
price                                                              2200.0
item_seq_number                                                       286
activation_date                       